# <center> Integrals with Intervals 

For a quick familiarization with the `ValidatedNumerics` package, the objective of this notebook is calculate an integral via Riemann sums, where two different implementations are defined using intervals. `Riemann_sums` and `Riemann_sums2` calculate the integral of a function in a given bound over a given number of partitions.


The idea is that the error `diam(Riemann_sums_n)` $\to$ `eps()` when `num_partitions` $\to \infty$ , we will be able to confirm that the integral is between the interval output of `Riemann_sums_n`.

**NOTE:** Because of the interval arithmetics, an overestimation of the diameter of the intervals is set for each operation made.

Let be $ \mathbf{f} : [a,b] \subset \mathbb{R} \to \mathbb{R} $ and $\mathbf{P} = \{a:\frac{b-a}{N}:b\}$ an homogeneous partition of $[a,b]$ of $N$ subintervals.

This way, we can define the **left Riemann sum** $S_l$ and **right Riemann sum** $S_r$ as
\begin{array}.
S_l &=& \sum_{i=1}^{n} f(x_{i-1})(x_{i}-x_{i-1})  \\
S_r &=& \sum_{i=1}^{n} f(x_i)(x_{i}-x_{i-1}).
\end{array}

The integral $I$, in case of existing, will be bounded by
$$ I \in \{ \min(S_l,S_r) , \max(S_l,S_r) \} $$ 
 
Thigther this bound, better the approximation.

In [1]:
using ValidatedNumerics

In [2]:
function Riemann_sums(f::Function,I::Vector,num_partitions::Int) 
    a, b = I
    dx = (b-a)/num_partitions
    S = 0 #initialize S variable
    for i in 1:num_partitions
        S += f(@interval(a + (i-1)*dx,a + i*dx))*dx
    end
    return S
end

Riemann_sums (generic function with 1 method)

In [3]:
function Riemann_sums2(f::Function,I::Vector,num_partitions::Int) 
    a, b = I
    dx = (b-a)/num_partitions
    dx_int = @interval(dx)
    S = 0 #initialize S variable
    for i in 1:num_partitions
        S += f(a + (i-0.5)*dx_int)*dx
    end
    return S
end

Riemann_sums2 (generic function with 1 method)

In [4]:
#Here one pases an Interval as the bounds
function Riemann_sums3(f::Function,I::Interval,num_partitions::Int)
    dx = @interval(diam(I)/num_partitions)
    S = 0
    for i in 1:num_partitions
        S += f(I.lo + (i - 0.5)*dx)*dx
    end
    return S
end

Riemann_sums3 (generic function with 1 method)

### Example 1: $f(x) = 2x$

We know that $ \int_{0}^{1} 2x \ dx = 1 $, so the computer's integral $I$, must be in the minimum interval containing 1, ie, `I = @interval(1)`

In [5]:
num_part = 20000

20000

In [6]:
f(x) = 2x 

f (generic function with 1 method)

In [29]:
bounds = [0,1]
@time S_f = Riemann_sums(f,bounds,num_part)

[0.9999499999994967, 1.0000500000005035]

1.015458 seconds (6.28 M allocations: 169.921 MB, 3.69% gc time)


In [30]:
@time S_f2 = Riemann_sums2(f,bounds,num_part)

[0.9999999999994968, 1.0000000000005034]

0.552987 seconds (3.18 M allocations: 86.668 MB, 3.97% gc time)


In [31]:
bounds_int = @interval(0,1)
@time S_f3 = Riemann_sums3(f,bounds_int,num_part)

[0.9999999999994968, 1.0000000000005034]

0.419374 seconds (2.66 M allocations: 71.716 MB, 4.56% gc time)


In [10]:
# ⪽ == ValidatedNumerics.interior::function
I_f = @interval(1)
I_f ⪽ S_f

true

In [11]:
I_f ⪽ S_f2

true

In [32]:
diam(I_f) , diam(S_f) , diam(S_f2), diam(S_f3)

(0.0,0.00010000000100673923,1.006528194125167e-12,1.006528194125167e-12)

In [13]:
#Comparison between `Riemann_sums` implementations 
S_f2 ⪽ S_f , S_f ⪽ S_f2 

0.511639 seconds (2.73 M allocations: 74.834 MB, 2.72% gc time)


(true,false)

### Example 2 $cos(x)$ 

$$ \int_0^{2\pi} cos(x) \ dx = 0 $$ 

In [33]:
bounds_cos = [0,2pi]
@time S_cos = Riemann_sums(cos,bounds_cos,num_part)

[-0.0006283185315984867, 0.0006283185315919266]

In [34]:
@time S_cos2 = Riemann_sums2(cos,bounds_cos,num_part)

1.590235 seconds (9.18 M allocations: 251.674 MB, 3.50% gc time)
  

[-8.685660155514918e-13, 8.675918598995136e-13]

In [35]:
bounds_cos_int = @interval(0,2pi)
@time S_cos3 = Riemann_sums3(cos,bounds_cos_int,num_part)

1.139085 seconds (6.20 M allocations: 170.252 MB, 3.53% gc time)
  

[-8.677866910299092e-13, 8.679705717183628e-13]

0.961119 seconds (5.78 M allocations: 156.521 MB, 3.45% gc time)


In [17]:
I_cos = @interval(0)
I_cos ⪽ S_cos

true

In [18]:
I_cos ⪽ S_cos2

true

In [19]:
diam(I_cos) , diam(S_cos) , diam(S_cos2) , diam(S_cos3)

(0.0,0.0012566370631904133,1.7361578754510054e-12,1.735757262748272e-12)

In [20]:
#Comparison between `Riemann_sums` implementations 
S_cos2 ⪽ S_cos , S_cos ⪽ S_cos2  

1.059278 seconds (5.78 M allocations: 156.521 MB, 3.33% gc time)


(true,false)

### Example 3: $G(x) = e^{-x^2} $

This is the Gaussian function which is not analytical, but it can be proved that 
$$ \int_{-\infty}^{\infty} e^{-x^2} dx = \sqrt{\pi} $$ 

In [21]:
Gauss(x) = exp(-x^2)
# obs: Gauss(|x|>28) == 0 for computer

Gauss (generic function with 1 method)

In [36]:
bounds_G = [-28,28]
@time S_gauss = Riemann_sums(Gauss,bounds_G,num_part)

[1.7696538509052664, 1.7752538509075266]

In [37]:
@time S_gauss2 = Riemann_sums2(Gauss,bounds_G,num_part)

1.110652 seconds (6.69 M allocations: 179.230 MB, 4.07% gc time)
  

[1.77245385090527, 1.7724538509075314]

0.582676 seconds (3.67 M allocations: 99.638 MB, 4.78% gc time)


In [38]:
bounds_G_int = @interval(-28,28)
@time S_gauss3 = Riemann_sums3(Gauss,bounds_G_int,num_part)

[1.77245385090527, 1.7724538509075314]

0.492822 seconds (3.25 M allocations: 88.043 MB, 4.43% gc time)


In [25]:
I_G = @interval(sqrt(pi))
I_G ⪽ S_gauss

true

In [26]:
I_G ⪽ S_gauss2

true

In [39]:
eps(), diam(I_G) , diam(S_gauss) , diam(S_gauss2), diam(S_gauss3)

(2.220446049250313e-16,4.440892098500626e-16,0.005600000002260241,2.261302256556519e-12,2.261302256556519e-12)

In [28]:
#Comparison between `Riemann_sums` implementations 
S_gauss2 ⪽ S_gauss , S_gauss ⪽ S_gauss2 

(true,false)

0.625183 seconds (3.25 M allocations: 88.043 MB, 3.97% gc time)


The interval integrator works without flaws and actually contains the real result of the integral within the interval limits. `Riemann_sums2` turned out to be much better than `Riemann_sums` in terms of time, memory and interval overestimation.  